<a href="https://colab.research.google.com/github/SantiagoGomezfpv/hyperparameter/blob/main/Grafos_GATConv_Optuna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Asegúrate de tener PyTorch Geometric y Optuna instalados en Colab.
!pip install torch torch_geometric optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 2.2 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GATConv
from torch_geometric.datasets import Planetoid
import optuna
import matplotlib.pyplot as plt

In [ ]:
# Cargar el conjunto de datos de ejemplo (Cora)
dataset = Planetoid(root='data/Cora', name='Cora')
data = dataset[0]

Processing...
Done!


# **Definir el Modelo de GAT Conv**

In [ ]:
class GAT(torch.nn.Module):
    def __init__(self, in_channels, out_channels, heads, concat, negative_slope, dropout, add_self_loops, edge_dim, fill_value, bias, residual):
        super(GAT, self).__init__()
        self.conv1 = GATConv(
            in_channels=in_channels,
            out_channels=out_channels,
            heads=heads,
            concat=concat,
            negative_slope=negative_slope,
            dropout=dropout,
            add_self_loops=add_self_loops,
            edge_dim=edge_dim,
            fill_value=fill_value,
            bias=bias,
            residual=residual
        )

        self.in_Channels_2 = (out_channels * heads if concat == True else out_channels)

        # Asegurar que conv2 tenga la dimensión correcta basada en la salida de conv1
        self.conv2 = GATConv(
            in_channels = self.in_Channels_2,
            out_channels = dataset.num_classes,
            heads=1,
            concat=False,
            dropout=dropout
        )

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# **Definir la Función de Entrenamiento**

In [ ]:
def train(model, data, optimizer):
    model.train()
    optimizer.zero_grad()
    output = model(data.x, data.edge_index)
    loss = F.nll_loss(output[data.train_mask], data.y[data.train_mask])  # Ajusta la pérdida según tu problema
    loss.backward()
    optimizer.step()
    return loss.item()

# **Definir la Función de Evaluación**

In [ ]:
def test(model, data):
    model.eval()
    with torch.no_grad():
        output = model(data.x, data.edge_index)
        pred = output.argmax(dim=1)
        correct = (pred[data.test_mask] == data.y[data.test_mask]).sum().item()
        accuracy = correct / data.test_mask.sum().item()
    return accuracy

# **Configurar la Función de Objetivo de Optuna**

In [ ]:
def objective(trial):
    try:
        # Hiperparámetros de GATConv a optimizar
        in_channels = dataset.num_features
        out_channels = trial.suggest_int("out_channels", 4, 32)
        heads = trial.suggest_int("heads", 1, 4)
        concat = trial.suggest_categorical("concat", [True, False])
        negative_slope = trial.suggest_float("negative_slope", 0.1, 0.3)
        dropout = trial.suggest_float("dropout", 0.0, 0.6)
        add_self_loops = trial.suggest_categorical("add_self_loops", [True, False])
        bias = trial.suggest_categorical("bias", [True, False])
        residual = trial.suggest_categorical("residual", [True, False])
        edge_dim = None
        fill_value = 'mean'

        # Crear el modelo con los hiperparámetros elegidos
        model = GAT(
            in_channels=in_channels,
            out_channels=out_channels,
            heads=heads,
            concat=concat,
            negative_slope=negative_slope,
            dropout=dropout,
            add_self_loops=add_self_loops,
            edge_dim=edge_dim,
            fill_value=fill_value,
            bias=bias,
            residual=residual
        )

        # Configurar el optimizador
        learning_rate = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

        # Entrenamiento y evaluación del modelo
        for epoch in range(100):
            train(model, data, optimizer)

        # Evaluar precisión en el conjunto de prueba
        accuracy = test(model, data)
        return accuracy

    except RuntimeError as e:
        # Si hay un error de dimensión, se registra el fallo y se devuelve None
        print(f"Trial fallido con error: {e}")
        return None

# **Ejecutar la Optimización de Hiperparámetros**

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

# Imprimir los mejores hiperparámetros encontrados
print("Mejores hiperparámetros:", study.best_params)
print("Mejor precisión:", study.best_value)

[I 2024-11-21 20:38:00,172] A new study created in memory with name: no-name-cfd2e882-d9a2-49de-9c33-9830d65d1778
[I 2024-11-21 20:38:07,398] Trial 0 finished with value: 0.784 and parameters: {'out_channels': 32, 'heads': 3, 'concat': True, 'negative_slope': 0.2733480465112832, 'dropout': 0.3888072084955692, 'add_self_loops': True, 'bias': True, 'residual': False, 'lr': 0.0037861900323756053}. Best is trial 0 with value: 0.784.
[I 2024-11-21 20:38:10,367] Trial 1 finished with value: 0.8 and parameters: {'out_channels': 14, 'heads': 1, 'concat': True, 'negative_slope': 0.26433166381994067, 'dropout': 0.2849413388529044, 'add_self_loops': True, 'bias': True, 'residual': False, 'lr': 0.003102393459560976}. Best is trial 1 with value: 0.8.
[I 2024-11-21 20:38:13,313] Trial 2 finished with value: 0.786 and parameters: {'out_channels': 9, 'heads': 2, 'concat': True, 'negative_slope': 0.17483803392578895, 'dropout': 0.5385143326214786, 'add_self_loops': False, 'bias': False, 'residual': Fal

Mejores hiperparámetros: {'out_channels': 30, 'heads': 2, 'concat': True, 'negative_slope': 0.12750270165229513, 'dropout': 0.5304567629726675, 'add_self_loops': True, 'bias': True, 'residual': True, 'lr': 0.001943496468471772}
Mejor precisión: 0.814


In [ ]:
best_params = study.best_params
model = GAT(dataset.num_features, dataset.num_classes, heads=best_params["heads"], dropout=best_params["dropout"])
optimizer = torch.optim.Adam(model.parameters(), lr=best_params["lr"])

for epoch in range(100):
    loss = train(model, data, optimizer)

accuracy = test(model, data)
print(f"Precisión final en el conjunto de prueba: {accuracy:.4f}")

TypeError: GAT.__init__() missing 7 required positional arguments: 'concat', 'negative_slope', 'add_self_loops', 'edge_dim', 'fill_value', 'bias', and 'residual'

In [ ]:
# Gráfico de historia de la optimización
optuna.visualization.plot_optimization_history(study).show()

# Gráfico de importancia de los hiperparámetros
optuna.visualization.plot_param_importances(study).show()